In [4]:
import os
import sys
import cv2
from matplotlib import pyplot as plt
import sys
import numpy as np
import face_alignment
import torch
from torch import nn

from masked_face_sdk.utils import end2end_mask_generation
from masked_face_sdk.pipeline_dataset_loader import PipelineFacesDatasetGenerator

from masked_face_sdk.pipeline_dataset_loader \
    import PipelineFacesDatasetGenerator
from masked_face_sdk.neural_network_modules \
    import Backbone, ArcFaceLayer, FaceRecognitionModel, resnet18

from torch.utils.data import DataLoader

%matplotlib inline

In [2]:
# Generate masks database
!python3 generate_masks_database.py \
    --masks-folder=data/masked_faces/ \
    --database-file=data/masks_base.json \
    --verbose --skip-warnings

100%|█████████████████████████████████████████| 244/244 [06:09<00:00,  1.51s/it]
Masks database successfully saved by follow path: data/masks_base.json


In [4]:
# Prepare training dataset
root_train_dataset_path = '/media/alexey/DataDisk/datasets/vggface2_with_face_boxes_markup/test_large/'

!python3 apply_masks_to_face_recognition_dataset.py \
    --face-dataset-folder={root_train_dataset_path} \
    --masks-database=data/masks_base.json \
    --verbose \
    --use-cuda

  0%|                                                   | 0/101 [00:00<?, ?it/s]Warning: No faces were detected.
Skipping and delete from base follow image: /media/alexey/DataDisk/datasets/vggface2_with_face_boxes_markup/test/n000001/0301_01.jpg, because can't find faces landmarks
 56%|███████████████████████▋                  | 57/101 [41:55<35:26, 48.32s/it]Warning: No faces were detected.
Skipping and delete from base follow image: /media/alexey/DataDisk/datasets/vggface2_with_face_boxes_markup/test/n001291/0445_01.jpg, because can't find faces landmarks
100%|███████████████████████████████████████| 101/101 [1:05:58<00:00, 39.19s/it]


In [ ]:
# Prepare test dataset
root_test_dataset_path = '/media/alexey/DataDisk/datasets/vggface2_with_face_boxes_markup/test_small/'

!python3 apply_masks_to_face_recognition_dataset.py \
    --face-dataset-folder={root_test_dataset_path} \
    --masks-database=data/masks_base.json \
    --verbose \
    --use-cuda

In [3]:
# Generate landmark estimator by face_alignment library
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cpu')

In [5]:
# Init constants

batch_size = 32
n_jobs = 4
epochs = 20
image_shape = (112, 112)
embedding_size = 128

In [6]:
# Init base variables for training

generator_train_dataset = PipelineFacesDatasetGenerator(
    root_train_dataset_path,
    image_shape
)

generator_test_dataset = PipelineFacesDatasetGenerator(
    root_train_dataset_path,
    image_shape
)

train_data = DataLoader(
        generator_train_dataset,
        batch_size=batch_size,
        num_workers=n_jobs,
        shuffle=True,
        drop_last=True
)

model = FaceRecognitionModel(
    backbone=Backbone(
        backbone=resnet18(pretrained=True),
        embedding_size=embedding_size,
        input_shape=(3, image_shape[0], image_shape[1])
    ),
    head=ArcFaceLayer(
        embedding_size=embedding_size,
        num_classes=generator_train_dataset.num_classes
    )
)

loss = torch.nn.CrossEntropyLoss()


NameError: name 'root_dataset_path' is not defined